In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
all_dir = []
x = 11
ppc = 16
for _, dirnames, filenames in os.walk('/media/alamin/0994E29B4C356F2D/Python/New life/1/12/image/'):
    all_dir.append(dirnames)
image = []
class_label = []
class_label2 = []
lavel_no = 0
for i in (all_dir[0]):
     for _, dirnames, filenames in os.walk('/media/alamin/0994E29B4C356F2D/Python/New life/1/12/image/'+i):
        for j in filenames:
            img = cv2.imread('/media/alamin/0994E29B4C356F2D/Python/New life/1/12/image/'+i+'/'+j)
            img = cv2.resize(img, (200, 200))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            laplacian = cv2.Laplacian(gray,cv2.CV_64F)
            sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=x) #13
            sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=x)
            sobelxy = cv2.Sobel(gray,cv2.CV_64F,1,1,ksize=x)
            equ = cv2.equalizeHist(gray)
            fd,hog_image = hog(equ, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
            
            ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create()
            gra = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            ridges = ridge_filter.getRidgeFilteredImage(equ)
            #ridges = ridge_filter.getRidgeFilteredImage(img)
            
            a = np.array(fd)
            #a = a.flatten()
            
            image.append(a)
            class_label.append(i)
            class_label2.append(lavel_no)
        lavel_no += 1
np.array(image).shape

/usr/local/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


(112, 10368)

In [3]:
class_label2 = np.array(class_label2)
image = np.array(image)

In [4]:
train_images, test_images, train_labels, test_labels  = train_test_split(image, class_label2, test_size=0.20, random_state=42)

In [5]:
model = keras.Sequential([
    keras.layers.Dense(20, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
89/89 [==============================] - 0s 3ms/step - loss: 1.3070 - acc: 0.4045
Epoch 2/5
89/89 [==============================] - 0s 214us/step - loss: 0.9414 - acc: 0.8202
Epoch 3/5
89/89 [==============================] - 0s 224us/step - loss: 0.6715 - acc: 0.9663
Epoch 4/5
89/89 [==============================] - 0s 248us/step - loss: 0.4689 - acc: 0.9888
Epoch 5/5
89/89 [==============================] - 0s 207us/step - loss: 0.3394 - acc: 1.0000


In [6]:
print(test_images.shape)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

(23, 10368)
23/23 [==============================] - 0s 2ms/step
Test accuracy: 1.0
